**input_vars**
7 times: 

* avg_cpu_util,
* slo_preservation,
* total_cpu_shares,
* cpu_shares_others,
* arrival_rate,
* latency

In [4]:
input_range_dict = {
    1: [(0.3, 0.6), (0.3, 0.6), (0.3, 0.6), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (0.3, 0.6)],
    2: [(0.2, 0.8), (0.2, 0.8), (0.2, 0.8), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (0.2, 0.8)],
    3: [(0.1, 1.0), (0.1, 1.0), (0.1, 1.0), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (0.1, 1.0)],
}

# Proporty Definition

## Robustness

In [2]:
DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"

def gen_prop(copy_one_argmax_candidate, copy_two_argmax_candidate, difficulty):
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    for i in range(2*model_output_size):
        lines.append(DECLARATION_LINE_BASE.format(f"Y_{i}"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range_dict[difficulty][i][0]))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range_dict[difficulty][i][1]))

    for i in range(model_input_size):
        if i % 8 in [0, 1, 2, 6]:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", -epsilon))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", 0))

    lines.append(" ")
    lines.append("; output constraints")
    for i in range(model_output_size):
        if i == copy_one_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{i}", f"Y_{copy_one_argmax_candidate}"))

    for i in range(model_output_size):
        if i == copy_two_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{model_output_size+i}", f"Y_{model_output_size+copy_two_argmax_candidate}"))
    
    path = base_path.format(difficulty, str(epsilon)[2:], copy_one_argmax_candidate, copy_two_argmax_candidate)

    f = open(path, "w")
    f.write('\n'.join(lines) )
    f.close()

In [3]:
model_input_size = 7
model_output_size = 6

epsilon = 0.001
tolerance = 0.40

base_path = "/home/mzi/sys-rl-verif/applications/firm/vnnlib/robustness/difficulty{}_eps{}_a{}_b{}.vnnlib"

for i in range(model_output_size):
    for j in range(model_output_size):
        if (i - j) > tolerance * model_output_size or (j-i) > tolerance * model_output_size:
            for difficulty in [1, 2, 3]:
                gen_prop(copy_one_argmax_candidate=i, copy_two_argmax_candidate=j, difficulty=difficulty)


## Monotonicity

In [2]:
DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"

def gen_prop(copy_one_argmax_candidate, copy_two_argmax_candidate, difficulty, feat_index):
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    for i in range(2*model_output_size):
        lines.append(DECLARATION_LINE_BASE.format(f"Y_{i}"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range_dict[difficulty][i][0]))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range_dict[difficulty][i][1]))

    for i in range(model_input_size):
        if i in feat_index:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", 0))

    lines.append(" ")
    lines.append("; output constraints")
    for i in range(model_output_size):
        if i == copy_one_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{i}", f"Y_{copy_one_argmax_candidate}"))

    for i in range(model_output_size):
        if i == copy_two_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{model_output_size+i}", f"Y_{model_output_size+copy_two_argmax_candidate}"))
    
    path = base_path.format(difficulty, str(epsilon)[2:], copy_one_argmax_candidate, copy_two_argmax_candidate)

    f = open(path, "w")
    f.write('\n'.join(lines) )
    f.close()

In [6]:
model_input_size = 7
model_output_size = 6

epsilon = 0.001
tolerance = 0.40
slo_preservation = [1]
# buffer_idx = [1]

base_path = "/home/mzi/sys-rl-verif/applications/firm/vnnlib/slo_preservation/difficulty{}_eps{}_a{}_b{}.vnnlib"
# base_path = "/home/mzi/sys-rl-verif/applications/firm/vnnlib/rebuffering_avoidance/difficulty{}_eps{}_a{}_b{}.vnnlib"
for i in range(model_output_size):
    for j in range(model_output_size):
        if (j-i) > tolerance * model_output_size:
            for difficulty in [1, 2, 3]:
                gen_prop(copy_one_argmax_candidate=i, copy_two_argmax_candidate=j, 
                    difficulty=difficulty, feat_index=slo_preservation)
